In [1]:
import pandas as pd
import pickle as pkl
import re
from collections import Counter
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 

import stanza
# stanza.download('en')
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma')

def count_freq(preds, gts):
    preds_len = [len(x.split(' ')) for x in preds]
    gts_len = [len(x.split(' ')) for x in gts]
    print(Counter(preds_len))
    print(Counter(gts_len))

def evaluation_metrics(pred, gt):
    TP = len(set(pred) & set(gt)) 
    FP = len(set(pred)-set(gt))
    FN = len(set(gt)-set(pred))
    precision = round((TP/(TP+FP))*100, 2)
    recall = round((TP/(TP+FN))*100,2)
    f1_score = round((2 * precision * recall) / (precision + recall),2)
    return precision, recall, f1_score 

def lemma(li):
    new_list = []
    for t in li:
        doc = nlp(str(t))
        doc1 = ' '.join([word.lemma for sent in doc.sentences for word in sent.words])
        doc1 = re.sub('-',' ',doc1)
        doc1 = re.sub(' +', ' ',doc1)
        new_list.append(doc1)
    new_list = [s for s in new_list if len(s) >= 2]
    return new_list

def get_term_(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for token, label in zip(tokens, labels):
            if label == 'B':
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
            # Check b_pos = 0 không
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms  

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hanhtran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2021-06-29 15:35:32 WARNING: Can not find mwt: default from official model list. Ignoring it.
2021-06-29 15:35:32 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |

2021-06-29 15:35:32 INFO: Use device: cpu
2021-06-29 15:35:32 INFO: Loading: tokenize
2021-06-29 15:35:32 INFO: Loading: pos
2021-06-29 15:35:33 INFO: Loading: lemma
2021-06-29 15:35:33 INFO: Done loading processors!


1. ADJ NOUN - 1

In [2]:
def get_term_adj_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'
        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if (b_pos != 0) and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' ') and (len(nlp(str(tokens[b_pos - 1])).sentences) > 0) and (len(nlp(str(tokens[b_pos])).sentences) > 0):
                    b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                    c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                    if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (b_word.text != 'None') and (b_word.upos == 'ADJ'):
                        terms.append(' '.join([b_word.text] + term))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

2. NOUN NOUN - 0

In [3]:
def get_term_noun_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0:
                    if (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' ') and (len(nlp(str(tokens[b_pos - 1])).sentences) > 0) and (len(nlp(str(tokens[b_pos])).sentences) > 0):
                        b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (b_word.text != 'None') and (b_word.upos == 'NOUN'):
                            terms.append(' '.join([b_word.text] + term))
                    if (tokens[i] != '') and (tokens[i] != ' ') and (len(nlp(str(tokens[i])).sentences) > 0):
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        if (a_word.text != 'None') and (a_word.upos == 'NOUN'):
                            terms.append(' '.join(term + [a_word.text]))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

3. ADJ NOUN NOUN - 1

In [4]:
def get_term_adj_noun_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                # Check b_pos = 0 không
                if b_pos != 0:
                    #ADJ NOUN NOUN
                    if (tokens[i] != '') and (tokens[i] != ' ') and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' ') and (len(nlp(str(tokens[b_pos - 1])).sentences) > 0) and (len(nlp(str(tokens[i])).sentences) > 0) and (len(nlp(str(tokens[b_pos])).sentences) > 0):
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        # Check vị trí b_pos - 1: terms.append()
                        if (c_word.text != 'None') and (c_word.upos == 'NOUN') and (b_word.text != 'None') and ((b_word.upos == 'ADJ') and (a_word.upos == 'NOUN')):
                            terms.append(' '.join([b_word.text] + term + [a_word.text]))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
            # check b_pos - 1
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

4. ADJ ADJ NOUN - 2

In [5]:
def get_term_adj_adj_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0:
                    #ADJ ADJ NOUN              
                    if (tokens[b_pos - 2] != '') and (tokens[b_pos - 2] != ' ') and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' ') and (len(nlp(str(tokens[b_pos - 2])).sentences) > 0 and len(nlp(str(tokens[b_pos - 1])).sentences) > 0 and len(nlp(str(tokens[b_pos])).sentences) > 0):
                        b1_word = nlp(str(tokens[b_pos - 2])).sentences[0].words[0] 
                        b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (b_word.text != 'None') and (b1_word.text != 'None') and ((b_word.upos == 'ADJ') and (b1_word.upos == 'ADJ')):
                            terms.append(' '.join([b1_word.text] +[b_word.text] + term))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

5. NOUN ADJ NOUN - 2

In [6]:
def get_term_noun_adj_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0:
                    #NOUN ADJ NOUN
                    if (tokens[b_pos - 2] != '') and (tokens[b_pos - 2] != ' ') and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' ') and (len(nlp(str(tokens[b_pos - 2])).sentences) > 0 and len(nlp(str(tokens[b_pos - 1])).sentences) > 0 and len(nlp(str(tokens[b_pos])).sentences) > 0):
                        b1_word = nlp(str(tokens[b_pos - 2])).sentences[0].words[0] 
                        b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (b_word.text != 'None') and (b1_word.text != 'None') and ((b_word.upos == 'ADJ') and (b1_word.upos == 'NOUN')):
                            terms.append(' '.join([b1_word.text] +[b_word.text] + term))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

6. NOUN NOUN NOUN - 0

In [7]:
def get_term_noun_noun_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                # NOUN NOUN NOUN
                if b_pos != 0 and i + 1 < len(tokens):
                    if (tokens[i] != '') and (tokens[i] != ' ') and (tokens[i+1] != '') and (tokens[i+1] != ' ') and (len(nlp(str(tokens[i])).sentences) > 0 and len(nlp(str(tokens[i+1])).sentences) > 0 and len(nlp(str(tokens[b_pos])).sentences) > 0):
                        a1_word = nlp(str(tokens[i+1])).sentences[0].words[0] 
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        if (c_word.text != 'None') and (c_word.upos == 'NOUN') and (a_word.text != 'None') and (a1_word.text != 'None') and ((a_word.upos == 'NOUN') and (a1_word.upos == 'NOUN')):
                            terms.append(' '.join( term + [a_word.text] +[a1_word.text]))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

7. NOUN ADP NOUN - 0

In [8]:
def get_term_noun_adp_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0 and i+1 < len(tokens):
                    if (tokens[i] != '') and (tokens[i] != ' ') and (tokens[i+1] != '') and (tokens[i+1] != ' ') and (len(nlp(str(tokens[i])).sentences) > 0 and len(nlp(str(tokens[i+1])).sentences) > 0 and len(nlp(str(tokens[b_pos])).sentences) > 0):
                        a1_word = nlp(str(tokens[i+1])).sentences[0].words[0] 
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        if (c_word.upos == 'NOUN')and (c_word.text != 'None') and (a_word.text != 'None') and (a1_word.text != 'None') and ((a_word.upos == 'ADP') and (a1_word.upos == 'NOUN')):
                            terms.append(' '.join(term + [a_word.text] + [a1_word.text]))                               
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower() for x in final_terms]
    return final_terms

8. NOUN ADP ADJ NOUN - 0

In [9]:
def get_term_noun_adp_adj_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())
        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'
        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0 and i + 2 < len(tokens):
                    if (tokens[i] != '') and (tokens[i] != ' ') and (tokens[i+1] != '') and (tokens[i+1] != ' ') and (tokens[i+2] != '') and (tokens[i+2] != ' ') and (tokens[b_pos] != '') and (tokens[b_pos] != ' ') and (len(nlp(str(tokens[i])).sentences) > 0 and len(nlp(str(tokens[i+1])).sentences) > 0 and len(nlp(str(tokens[i+2])).sentences) > 0 and len(nlp(str(tokens[b_pos])).sentences) > 0):
                        a2_word = nlp(str(tokens[i+2])).sentences[0].words[0] 
                        a1_word = nlp(str(tokens[i+1])).sentences[0].words[0] 
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (a_word.text != 'None') and (a1_word.text != 'None') and (a2_word.text != 'None') and (a_word.upos == 'ADP') and (a1_word.upos == 'ADJ') and (a2_word.upos == 'NOUN'):
                            terms.append(' '.join(term + [a_word.text] + [a1_word.text] +[a2_word.text]))   
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower() for x in final_terms]
    return final_terms    

9. ADJ NOUN ADP NOUN - 0

In [10]:
def get_term_adj_noun_adp_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0 and i + 1 < len(tokens):
                    if (tokens[i+1] != '') and (tokens[i+1] != ' ') and (tokens[i] != '') and (tokens[i] != ' ') and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' ') and len(nlp(str(tokens[b_pos - 1])).sentences) > 0 and (len(nlp(str(tokens[i])).sentences) > 0) and (len(nlp(str(tokens[i+1])).sentences) > 0) and (len(nlp(str(tokens[b_pos])).sentences) > 0):
                        a1_word = nlp(str(tokens[i+1])).sentences[0].words[0]
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        if (c_word.text != 'None') and (c_word.upos == 'NOUN') and (b_word.text != 'None') and (a_word.text != 'None') and (a1_word.text != 'None') and (b_word.upos == 'ADJ') and (a_word.upos == 'ADP') and (a1_word.upos == 'NOUN'):
                            terms.append(' '.join([b_word.text] + term + [a_word.text] + [a1_word.text]))        
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower() for x in final_terms]
    return final_terms  

10. NOUN ADP NOUN NOUN - 0

In [11]:
def get_term_noun_adp_noun_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if b_pos != 0 and i + 2 < len(tokens):
                    if (tokens[i+2] != '') and (tokens[i+2] != ' ') and (tokens[i+1] != '') and (tokens[i+1] != ' ') and (tokens[i] != '') and (tokens[i] != ' ') and (len(nlp(str(tokens[i])).sentences) > 0) and (len(nlp(str(tokens[i+1])).sentences) > 0) and (len(nlp(str(tokens[i+2])).sentences) > 0) and (len(nlp(str(tokens[b_pos])).sentences) > 0):
                            a2_word = nlp(str(tokens[i+2])).sentences[0].words[0]
                            a1_word = nlp(str(tokens[i+1])).sentences[0].words[0]
                            a_word = nlp(str(tokens[i])).sentences[0].words[0]
                            c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                            if (c_word.text != 'None') and (c_word.upos == 'NOUN') and (a_word.text != 'None')and (a1_word.text != 'None') and (a2_word.text != 'None') and (a_word.upos == 'ADP') and (a1_word.upos == 'NOUN') and (a2_word.upos == 'NOUN'):
                                terms.append(' '.join(term + [a_word.text] + [a1_word.text] + [a2_word.text]))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
            # check b_pos - 1
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower() for x in final_terms]
    return final_terms  

11. ADJ NOUN ADP ADJ NOUN - 1

In [12]:
def get_term_adj_noun_adp_adj_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if (b_pos != 0) and (i + 2 < len(tokens)) and (tokens[i+2] != '') and (tokens[i+2] != ' ') and (tokens[i+1] != '') and (tokens[i+1] != ' ') and (tokens[i] != '') and (tokens[i] != ' ')and (tokens[b_pos] != '') and (tokens[b_pos] != ' ') and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' ') and (len(nlp(str(tokens[b_pos - 1])).sentences) > 0) and (len(nlp(str(tokens[i])).sentences) > 0) and (len(nlp(str(tokens[i+1])).sentences) > 0) and (len(nlp(str(tokens[i+2])).sentences) > 0) and (len(nlp(str(tokens[b_pos])).sentences) > 0):
                    a2_word = nlp(str(tokens[i+2])).sentences[0].words[0]
                    a1_word = nlp(str(tokens[i+1])).sentences[0].words[0]
                    a_word = nlp(str(tokens[i])).sentences[0].words[0]
                    b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                    c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                    if (c_word.text != 'None') and (c_word.upos == 'NOUN') and (b_word.text != 'None') and (a_word.text != 'None') and (a1_word.text != 'None') and (a2_word.text != 'None') and (b_word.upos == 'ADJ') and (a_word.upos == 'ADP') and (a1_word.upos == 'ADJ')and (a2_word.upos == 'NOUN'):
                        terms.append(' '.join([b_word.text] + term + [a_word.text] + [a1_word.text]  + [a2_word.text]))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower() for x in final_terms]
    return final_terms  

12. NOUN PRON - 0

In [13]:
def get_term_noun_pron(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if (b_pos != 0) and (tokens[i] != '') and (tokens[i] != ' ') and (tokens[b_pos] != '') and (tokens[b_pos] != ' ') and (len(nlp(str(tokens[i])).sentences) > 0) and (len(nlp(str(tokens[b_pos])).sentences) > 0):
                        a_word = nlp(str(tokens[i])).sentences[0].words[0]
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        # Check vị trí b_pos - 1: terms.append()
                        if (c_word.text != 'None') and (c_word.upos == 'NOUN')  and (a_word.text != 'None') and (a_word.upos == 'PROPN'):
                            terms.append(' '.join(term + [a_word.text]))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
            # check b_pos - 1
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower() for x in final_terms]
    return final_terms  

13. ADV ADJ NOUN - 2

In [14]:
def get_term_adv_adj_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'
        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if (b_pos != 0) and (tokens[b_pos - 2] != '') and (tokens[b_pos - 2] != ' ') and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' ') and (len(nlp(str(tokens[b_pos - 2])).sentences) > 0 and len(nlp(str(tokens[b_pos - 1])).sentences) > 0 and len(nlp(str(tokens[b_pos])).sentences) > 0):
                        b1_word = nlp(str(tokens[b_pos - 2])).sentences[0].words[0] 
                        b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                        c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                        if (c_word.text != 'None') and (c_word.upos == 'NOUN') and (b_word.text != 'None') and (b1_word.text != 'None') and ((b_word.upos == 'ADJ') and (b1_word.upos == 'ADV')):
                            terms.append(' '.join([b1_word.text] +[b_word.text] + term))                   
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
        
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower() for x in final_terms]
    return final_terms    

14. ADJ NOUN ADJ NOUN - 1

In [15]:
def get_term_adj_noun_adj_noun(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                if (b_pos != 0) and (i + 1 < len(tokens)) and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' ') and (tokens[b_pos] != '') and (tokens[b_pos] != ' ') and (tokens[i] != '') and (tokens[i] != ' ') and (tokens[i+1] != '') and (tokens[i+1] != ' ') and (len(nlp(str(tokens[b_pos - 1])).sentences) > 0) and (len(nlp(str(tokens[b_pos])).sentences) > 0) and (len(nlp(str(tokens[i])).sentences) > 0) and  (len(nlp(str(tokens[i+1])).sentences) > 0): 
                    a1_word = nlp(str(tokens[i+1])).sentences[0].words[0]
                    a_word = nlp(str(tokens[i])).sentences[0].words[0]
                    b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                    c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                    # Check vị trí b_pos - 1: terms.append()
                    if (c_word.text != 'None') and (b_word.text != 'None') and (a_word.text != 'None') and (a1_word.text != 'None') and (c_word.upos == 'NOUN') and ((b_word.upos == 'ADJ') and (a_word.upos == 'ADJ') and (a1_word.upos == 'NOUN')):
                        terms.append(' '.join([b_word.text] + term + [a_word.text] + [a1_word.text]))
                                
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
            # check b_pos - 1
        
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower() for x in final_terms]
    return final_terms

In [16]:
def term_evaluation(domain_path, preds_path, rule=None):
    groundtruth = pd.read_csv(domain_path, sep='	', engine='python',header=None)
    gt = list(groundtruth[0])
    predictions = pkl.load(open(preds_path, 'rb'))
    if rule == 'adj_noun':
        preds =  get_term_adj_noun(predictions)
    elif rule == 'noun_noun':
        preds =  get_term_noun_noun(predictions)
    elif rule == 'adj_adj_noun':
        preds = get_term_adj_adj_noun(predictions)
    elif rule == 'adj_noun_noun':
        preds = get_term_adj_noun_noun(predictions)
    elif rule == 'noun_adj_noun':
        preds = get_term_noun_adj_noun(predictions)
    elif rule == 'noun_noun_noun':
        preds = get_term_noun_noun_noun(predictions)
    elif rule == 'noun_adp_noun':
        preds = get_term_noun_adp_noun(predictions)
    elif rule == 'noun_adp_adj_noun':
        preds = get_term_noun_adp_adj_noun(predictions)
    elif rule == 'adj_noun_adp_noun':
        preds = get_term_adj_noun_adp_noun(predictions)
    elif rule == 'noun_adp_noun_noun':
        preds = get_term_noun_adp_noun_noun(predictions)
    elif rule == 'adj_noun_adp_adj_noun':
        preds = get_term_adj_noun_adp_adj_noun(predictions)
    elif rule == 'adv_adj_noun':
        preds = get_term_adv_adj_noun(predictions)
    elif rule == 'noun_pron':
        preds = get_term_noun_pron(predictions)
    elif rule == 'adj_noun_adj_noun':
        preds = get_term_adj_noun_adj_noun(predictions)
    else:
        preds =  get_term_(predictions)
    stop_words = set(stopwords.words('english'))
    pred_terms =  set(preds) - set(stop_words)
    pred_terms = [x for x in pred_terms if len(x)>1]
    pred_terms = [x.lower().strip() for x in pred_terms]
    pred_terms = [re.sub(' -','-', x) for x in pred_terms]
    pred_terms = [re.sub('- ','-', x) for x in pred_terms]
    pred_terms = [re.sub('\(','', x) for x in pred_terms]
    pred_terms = [re.sub('\/','', x) for x in pred_terms]
    precision, recall, f1 = evaluation_metrics(pred_terms, gt)
    return precision, recall, f1
    

In [17]:
path = '/Users/hanhtran/Documents/terminology-extraction/ACTER/en/'
path1 = '/Users/hanhtran/Documents/terminology-extraction/patterns/en_domains/corp_wind/'
domain_paths = ['equi/annotations/equi_en_terms.ann', 'equi/annotations/equi_en_terms_nes.ann']
preds_paths = ['ann_xlnet_equi.pkl','nes_xlnet_equi.pkl']
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(path + d,path1 + p)
    results.append([p, f1])
raw_res = pd.DataFrame(results,columns=['DOMAIN','NON_PATTERN'])
raw_res

,DOMAIN,NON_PATTERN
0,ann_xlnet_equi.pkl,34.81
1,nes_xlnet_equi.pkl,40.62


In [18]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(path + d,path1 + p, 'adj_noun')
    results.append([p,f1])
adj_noun = pd.DataFrame(results,columns=['DOMAIN','ADJ_NOUN'])
adj_noun

,DOMAIN,ADJ_NOUN
0,ann_xlnet_equi.pkl,33.63
1,nes_xlnet_equi.pkl,39.64


In [19]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1  = term_evaluation(path + d,path1 + p, 'noun_noun')
    results.append([p, f1])
noun_noun = pd.DataFrame(results,columns=['DOMAIN','NOUN_NOUN'])
noun_noun

,DOMAIN,NOUN_NOUN
0,ann_xlnet_equi.pkl,32.74
1,nes_xlnet_equi.pkl,38.51


In [20]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(path + d,path1 + p, 'adj_noun_noun')
    results.append([p, f1])
adj_noun_noun = pd.DataFrame(results,columns=['DOMAIN','ADJ_NOUN_NOUN'])
adj_noun_noun

,DOMAIN,ADJ_NOUN_NOUN
0,ann_xlnet_equi.pkl,34.60
1,nes_xlnet_equi.pkl,40.39


In [21]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(path + d,path1 + p, 'adj_adj_noun')
    results.append([p, f1])
adj_adj_noun = pd.DataFrame(results,columns=['DOMAIN','ADJ_ADJ_NOUN'])
adj_adj_noun

,DOMAIN,ADJ_ADJ_NOUN
0,ann_xlnet_equi.pkl,34.68
1,nes_xlnet_equi.pkl,40.52


In [22]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(path + d,path1 + p, 'noun_adj_noun')
    results.append([p, f1])
noun_adj_noun = pd.DataFrame(results,columns=['DOMAIN','NOUN_ADJ_NOUN'])
noun_adj_noun

,DOMAIN,NOUN_ADJ_NOUN
0,ann_xlnet_equi.pkl,34.74
1,nes_xlnet_equi.pkl,40.52


In [23]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(path + d,path1 + p, 'noun_noun_noun')
    results.append([p,f1])
noun_noun_noun = pd.DataFrame(results,columns=['DOMAIN','NOUN_NOUN_NOUN'])
noun_noun_noun

,DOMAIN,NOUN_NOUN_NOUN
0,ann_xlnet_equi.pkl,34.73
1,nes_xlnet_equi.pkl,40.57


In [24]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(path + d,path1 + p, 'noun_adp_noun')
    results.append([p, f1])
noun_adp_noun = pd.DataFrame(results,columns=['DOMAIN','NOUN_ADP_NOUN'])
noun_adp_noun

,DOMAIN,NOUN_ADP_NOUN
0,ann_xlnet_equi.pkl,34.05
1,nes_xlnet_equi.pkl,39.81


In [25]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(path + d,path1 + p, 'noun_adp_adj_noun')
    results.append([p,f1]) 
noun_adp_adj_noun = pd.DataFrame(results,columns=['DOMAIN','NOUN_ADP_ADJ_NOUN'])
noun_adp_adj_noun

,DOMAIN,NOUN_ADP_ADJ_NOUN
0,ann_xlnet_equi.pkl,34.61
1,nes_xlnet_equi.pkl,40.40


In [26]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(path + d,path1 + p, 'noun_adp_noun_noun')
    results.append([p, f1])
noun_adp_noun_noun = pd.DataFrame(results,columns=['DOMAIN','NOUN_ADP_NOUN_NOUN'])
noun_adp_noun_noun

,DOMAIN,NOUN_ADP_NOUN_NOUN
0,ann_xlnet_equi.pkl,34.68
1,nes_xlnet_equi.pkl,40.53


In [27]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(path + d,path1 + p, 'adj_noun_adp_adj_noun')
    results.append([p, f1])
adj_noun_adp_adj_noun = pd.DataFrame(results,columns=['DOMAIN','ADJ_NOUN_ADP_ADJ_NOUN'])
adj_noun_adp_adj_noun

,DOMAIN,ADJ_NOUN_ADP_ADJ_NOUN
0,ann_xlnet_equi.pkl,34.78
1,nes_xlnet_equi.pkl,40.58


In [28]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(path + d,path1 + p, 'adv_adj_noun')
    results.append([p, f1])
adv_adj_noun = pd.DataFrame(results,columns=['DOMAIN', 'ADV_ADJ_NOUN'])
adv_adj_noun

,DOMAIN,ADV_ADJ_NOUN
0,ann_xlnet_equi.pkl,34.64
1,nes_xlnet_equi.pkl,40.36


In [29]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(path + d,path1 + p, 'noun_pron')
    results.append([p, f1])
noun_pron = pd.DataFrame(results,columns=['DOMAIN','NOUN_PRON'])
noun_pron

,DOMAIN,NOUN_PRON
0,ann_xlnet_equi.pkl,34.6
1,nes_xlnet_equi.pkl,40.5


In [30]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(path + d,path1 + p, 'adj_noun_adj_noun')
    results.append([p, f1])
adj_noun_adj_noun = pd.DataFrame(results,columns=['DOMAIN','ADJ_NOUN_ADJ_NOUN'])
adj_noun_adj_noun

,DOMAIN,ADJ_NOUN_ADJ_NOUN
0,ann_xlnet_equi.pkl,34.80
1,nes_xlnet_equi.pkl,40.61


In [31]:
results = []
for d, p in zip(domain_paths, preds_paths):
    pre, rec, f1 = term_evaluation(path + d,path1 + p, 'adj_noun_adp_noun')
    results.append([p,f1])
adj_noun_adp_noun = pd.DataFrame(results,columns=['DOMAIN','ADJ_NOUN_ADP_NOUN'])
adj_noun_adp_noun

,DOMAIN,ADJ_NOUN_ADP_NOUN
0,ann_xlnet_equi.pkl,34.81
1,nes_xlnet_equi.pkl,40.58


In [32]:
df_ls = [adj_noun, noun_noun, adj_adj_noun, adj_noun_noun, noun_adj_noun, 
        noun_noun_noun , noun_adp_noun, noun_adp_adj_noun, adj_noun_adp_noun, noun_adp_noun_noun,
        adj_noun_adp_adj_noun, adv_adj_noun, noun_pron, adj_noun_adj_noun]

In [33]:
for d in df_ls:
    raw_res = raw_res.merge(d, on='DOMAIN', how='left')
raw_res

,DOMAIN,NON_PATTERN,ADJ_NOUN,NOUN_NOUN,ADJ_ADJ_NOUN,ADJ_NOUN_NOUN,NOUN_ADJ_NOUN,NOUN_NOUN_NOUN,NOUN_ADP_NOUN,NOUN_ADP_ADJ_NOUN,ADJ_NOUN_ADP_NOUN,NOUN_ADP_NOUN_NOUN,ADJ_NOUN_ADP_ADJ_NOUN,ADV_ADJ_NOUN,NOUN_PRON,ADJ_NOUN_ADJ_NOUN
0,ann_xlnet_equi.pkl,34.81,33.63,32.74,34.68,34.60,34.74,34.73,34.05,34.61,34.81,34.68,34.78,34.64,34.6,34.80
1,nes_xlnet_equi.pkl,40.62,39.64,38.51,40.52,40.39,40.52,40.57,39.81,40.40,40.58,40.53,40.58,40.36,40.5,40.61


In [34]:
raw_res.to_csv('en_iate_f1.csv')

In [35]:
# cols = [col for col in raw_res.columns if 'DOMAIN' in col or '_F1' in col]
# raw_res[cols]

In [36]:
# f1_df = raw_res[cols].set_index('DOMAIN').T
# f1_df.to_csv('equi_wind/en_iate_f1.csv') #corp_equi

In [37]:
# def test_term_evaluation(domain_path, preds_path, rule=None):
#     groundtruth = pd.read_csv(domain_path, sep='	', engine='python',header=None)
#     gt = list(groundtruth[0])
#     predictions = pkl.load(open(preds_path, 'rb'))
#     if rule == 'adj_noun':
#         preds =  get_term_adj_noun(predictions)
#     elif rule == 'noun_noun':
#         preds =  get_term_noun_noun(predictions)
#     elif rule == 'adj_adj_noun':
#         preds = get_term_adj_adj_noun(predictions)
#     elif rule == 'adj_noun_noun':
#         preds = get_term_adj_noun_noun(predictions)
#     elif rule == 'noun_adj_noun':
#         preds = get_term_noun_adj_noun(predictions)
#     elif rule == 'noun_noun_noun':
#         preds = get_term_noun_noun_noun(predictions)
#     elif rule == 'noun_adp_noun':
#         preds = get_term_noun_adp_noun(predictions)
#     elif rule == 'noun_adp_adj_noun':
#         preds = get_term_noun_adp_adj_noun(predictions)
#     elif rule == 'adj_noun_adp_noun':
#         preds = get_term_adj_noun_adp_noun(predictions)
#     elif rule == 'noun_adp_noun_noun':
#         preds = get_term_noun_adp_noun_noun(predictions)
#     elif rule == 'adj_noun_adp_adj_noun':
#         preds = get_term_adj_noun_adp_adj_noun(predictions)
#     elif rule == 'adv_adj_noun':
#         preds = get_term_adv_adj_noun(predictions)
#     elif rule == 'noun_pron':
#         preds = get_term_noun_pron(predictions)
#     elif rule == 'adj_noun_adj_noun':
#         preds = get_term_adj_noun_adj_noun(predictions)
#     else:
#         preds =  get_term_(predictions)
#     stop_words = set(stopwords.words('english'))
#     pred_terms =  set(preds) - set(stop_words)
#     pred_terms = [x for x in pred_terms if len(x)>1]
#     pred_terms = [x.lower().strip() for x in pred_terms]
#     pred_terms = [re.sub(' -','-', x) for x in pred_terms]
#     pred_terms = [re.sub('- ','-', x) for x in pred_terms]
#     pred_terms = [re.sub('\(','', x) for x in pred_terms]
#     pred_terms = [re.sub('\/','', x) for x in pred_terms]
#     precision, recall, f1 = evaluation_metrics(pred_terms, gt)
#     return gt, pred_terms
    

In [38]:
# gt_w, preds_w = test_term_evaluation(path + domain_paths[1], path1 + preds_paths[1])

In [39]:
# gt_w[:20]

In [40]:
# preds_w[:20]

In [41]:
# gt_w1, preds_w1 = test_term_evaluation(path + domain_paths[1], path1 + preds_paths[1], 'adj_noun')

In [42]:
# preds_w1[:20]

In [43]:
# len(set(gt_w) - set(preds_w))

In [44]:
# len(set(gt_w) - set(preds_w1))